# Paquetería necesaria

In [ ]:
using PyPlot
using Interact
using LaTeXStrings

##Llamar a archivos necesarios para trabajar

In [ ]:
L = 12
# = "WL2D"
#geometria = "triangular"
#modelo = "Is_antiferr"
caracterizacion = "WL2D_triangular_L_$(L)_Is_ferr_cam_10_f_0.0001"
#datos = readdlm("Entropia_2D_final_$(algoritmo)_$(geometria)_L_$(L)_$(modelo).txt")
datos = readdlm("Entropia_2D_final_$(caracterizacion).txt")

In [ ]:
sistema =  readdlm("Datos_del_sistema_$(caracterizacion).txt")

#Creando variables y arreglos necesarios 

In [ ]:
temperatura = [.4:.1:8]
β = 1./temperatura
h = [0:.2:4]
m = length(β)
n = length(h)
l = size(datos)[1];

In [ ]:
constante_de_renormalización = [0.0 for i in 1:length(β), j in 1:length(h)]
energía_promedio = [0.0 for i in 1:length(β), j in 1:length(h)]
magnetización_promedio = [0.0 for i in 1:length(β), j in 1:length(h)]
parámetro_de_orden_promedio = [0.0 for i in 1:length(β), j in 1:length(h)]
capacidad_calorífica = [0.0 for i in 1:length(β), j in 1:length(h)]
susceptibilidad_magnética = [0.0 for i in 1:length(β), j in 1:length(h)];

#Creando funciones

In [ ]:
function Renormalizar(i,j)
    for k in 1:l
        if constante_de_renormalización[i,j]<=datos[k,3]-β[i]*(datos[k,1]-h[j]*datos[k,2])
            constante_de_renormalización[i,j]=datos[k,3]-β[i]*(datos[k,1]-h[j]*datos[k,2])
        end
    end
end 

In [ ]:
function Rellenar_matriz_de_renormalización()
    for i=1:length(β) 
        for j=1:length(h)
            Renormalizar(i,j)
        end
    end
end

In [ ]:
Rellenar_matriz_de_renormalización()

In [ ]:
function Variables_Macroscópicas(i,j)
    función_de_partición_1D = 0.0
    función_de_partición_2D = 0.0
    energía = 0.0
    energía_cuadrada = 0.0
    magnetización = 0.0
    magnetización_cuadrada = 0.0
    parámetro_de_orden = 0.0
    exponente = 0.0
    for k in 1:l
        exponente = datos[k,3] - β[i]*(datos[k,1]-h[j]*datos[k,2]) - constante_de_renormalización[i,j]
        función_de_partición_2D += exp(exponente)
        energía += abs(datos[k,1])*exp(exponente)
        energía_cuadrada += datos[k,1]*datos[k,1]*exp(exponente)
        magnetización += abs(datos[k,2])*exp(exponente)
        magnetización_cuadrada += datos[k,2]*datos[k,2]*exp(exponente)
        parámetro_de_orden += abs(datos[k,4])*exp(exponente)
    end
    energía_promedio[i,j] = (energía)/(función_de_partición_2D*sistema[2])
    magnetización_promedio[i,j] = (magnetización)/(función_de_partición_2D*sistema[2])
    parámetro_de_orden_promedio[i,j] = (parámetro_de_orden)/(función_de_partición_2D*sistema[2])
    capacidad_calorífica[i,j] = abs(energía_cuadrada - energía*energía)/(función_de_partición_2D*sistema[2]*sistema[2])
    susceptibilidad_magnética[i,j] = abs(magnetización_cuadrada - magnetización*magnetización)/(función_de_partición_2D*sistema[2]*sistema[2])
end

In [ ]:
function Obtener_variables_macroscópicas()
    for i=1:m
        for j=1:n
            Variables_Macroscópicas(i,j)
        end
    end
end

##Obtener y graficar variables macroscópicas

In [ ]:
Obtener_variables_macroscópicas()

In [ ]:
fig = figure()

@manipulate for i in 1:n
    withfig(fig) do
        subplot(221)
        ylim(0,1.05)
        xlabel(L"Temperatura")
        ylabel(L"P.O.")
        plot(temperatura,parámetro_de_orden_promedio[:,i], label = latexstring("h=$(h[j])"))
        legend()
        subplot(222)
        xlabel(L"Temperatura")
        ylabel(L"Energía")
        plot(temperatura,energía_promedio[:,i], label = latexstring("h=$(h[j])"))
        legend()
        subplot(223)
        xlabel(L"Temperatura")
        ylabel(L"Capacidad\ calorífica")
        plot(temperatura,capacidad_calorífica[:,i], label = latexstring("h=$(h[j])"))
        legend()
        subplot(224)
        xlabel(L"Temperatura")
        ylabel(L"Susceptibilidad\ magnética")
        plot(temperatura,susceptibilidad_magnética[:,i], label = latexstring("h=$(h[j])"))
        legend()
    end
end

In [ ]:
function grafica_magnetizacion_h_fija(j)
    ylim(0,1.05)
    xlabel(L"Temperatura")
    ylabel(L"P.O.")
    title(L"Parámetro\ de\ orden")
    plot(temperatura,parámetro_de_orden_promedio[:,j], label = latexstring("h=$(h[j])"))
    legend()
end

In [ ]:
function grafica_observable_macroscópica_con_h_fija(observable_macroscópica, cadena)
    xlabel(L"Temperatura")
    ylabel(cadena)
    title(cadena)
    plot(temperatura,observable_macroscópica, label = latexstring("h=$(h[j])"))
    legend()
end

#;open Magnetizacion_$(caracterizacion).pdf

In [ ]:
j=4
grafica_observable_macroscópica_con_h_fija(capacidad_calorífica[:,j], L"Capacidad\ calorífica")
savefig("Capacidad_calorifica_$(caracterizacion)_h_$(int(h[j]*10))E-1.pdf")

In [ ]:
entrada = 1
grafica_magnetizacion_h_fija(entrada)
savefig("Magnetizacion_$(caracterizacion)_h_$(int(h[entrada]*10))E-1.pdf")

In [ ]:

for i in 1:n
        ylim(0,1.05)
    plot(temperatura,magnetización_promedio[:,i],label = "h=$i")
    #legend (loc="upper right")
    xlabel (L"Temperatura")
    ylabel (L"Parámetro\ de\ orden")
end
#text(1.,.5,L"h=2")

##Graficar entropía exacta (si la hay)

In [ ]:
function grafica3d(angle,x,y,z)
    plot3D(x,y,z, "o", alpha=0.2 )
        ax = gca()
        ax[:view_init](elev=30., azim=angle)
end

In [ ]:
fig = figure(figsize=(8,6))
@manipulate for angle=0:10:360
    withfig(fig) do
        grafica3d(angle,datos[:,1],datos[:,2],datos[:,3])
    end
end

In [ ]:
function grafica3d_exact(angle)
    plot3D(exact[:2], exact[:3], exact[:1], "o", alpha=0.2 )
        ax = gca()
        ax[:view_init](elev=30., azim=angle)
end

##Graficar entropía simulada

In [ ]:
function grafica3d(angle,x,y,z)
    plot3D(x,y,z, "o", alpha=0.2 )
        ax = gca()
        ax[:view_init](elev=30., azim=angle)
end

In [ ]:
function grafica3d_entropia(angle)
    plot3D(datos[:,1],datos[:,2],datos[:,3], "o", alpha=0.2 )
        ax = gca()
        ax[:view_init](elev=30., azim=angle)
end

In [ ]:
fig = figure(figsize=(5,4))
@manipulate for angle=0:10:360
    withfig(fig) do
        surf(temperatura, h, magnetización_promedio', alpha=0.2, rstride=3, cstride=3)
        ax = gca()
        ax[:view_init](elev=30., azim=angle)
        xlabel(L"T")
        ylabel(L"h")
        zlabel(L"M(T,h)")
    end
end

In [ ]:
fig = figure(figsize=(8,6))
@manipulate for angle=0:10:360
    withfig(fig) do
        grafica3d_entropia(angle)
    end
end
#savefig("hola.pdf")

In [ ]:
grafica3d_entropia(220)
savefig("Entropia_2D_$(caracterizacion).pdf")

In [ ]:
;open Entropia_2D_$(caracterizacion).pdf

In [ ]:
datos

In [ ]:
exact = readdlm("ising_c16xc16.txt")

In [ ]:
exact[:,2]=exact[:,2]*(-1)

In [ ]:
exact

In [ ]:
function grafica3d_exact(angle)
    plot3D(exact[:,2], exact[:,3], log(exact[:,1]), "o", alpha=0.2 )
        ax = gca()
        ax[:view_init](elev=30., azim=angle)
end

In [ ]:
map(x -> x^2,h); # x ↦ x²

In [ ]:
exact = sortrows(exact, by=x->(x[2],x[3]))

In [ ]:
grafica3d_exact(220)
savefig("entropia_exacta_L_$L.pdf")

In [ ]:
fig = figure(figsize=(8,6))
@manipulate for angle=0:10:360
#    @manipulate for latitud=-70:10:70
    withfig(fig) do
        plot3D(datos[:,1], datos[:,2], datos[:,3]-log(exact[:,1]), "o", alpha=0.2 )
        ax = gca()
            ax[:view_init](elev=30., azim=angle)
#    end
    end
end


In [ ]:
plot3D(datos[:,1], datos[:,2], datos[:,3]-log(exact[:,1]), "o", alpha=0.2 )
        ax = gca()
            ax[:view_init](elev=30., azim=220)
savefig("entropia_exacta_vs_entropia_sim_L_$L.pdf")